In [ ]:
# Cell 1: Imports and data generation (BUGGY NOTEBOOK)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import warnings
warnings.filterwarnings("ignore")

# Synthetic loan dataset
np.random.seed(42)
n_samples = 1000

income = np.random.lognormal(mean=10.5, sigma=0.5, size=n_samples)
credit_score = np.random.normal(loc=680, scale=60, size=n_samples).clip(300, 850)
loan_amount = np.random.lognormal(mean=10.0, sigma=0.6, size=n_samples)
term_months = np.random.choice([36, 60], size=n_samples, p=[0.6, 0.4])
dti_ratio = np.random.uniform(0.05, 0.5, size=n_samples)

education_level = np.random.choice(
    ["High School", "Bachelors", "Masters", "PhD"], size=n_samples, p=[0.3, 0.4, 0.25, 0.05]
)
loan_purpose = np.random.choice(
    ["Debt Consolidation", "Home Improvement", "Car", "Medical", "Vacation", "Other"],
    size=n_samples
)

# Default probability synthesis
base = (
    0.15
    + 0.25 * (credit_score < 620).astype(int)
    + 0.20 * (dti_ratio > 0.35).astype(int)
    + 0.15 * (loan_amount > np.median(loan_amount)).astype(int)
    + 0.10 * (term_months == 60).astype(int)
)
noise = 0.05 * np.random.randn(n_samples)
default_prob = np.clip(base + noise, 0, 0.95)
default = (np.random.rand(n_samples) < default_prob).astype(int)

df = pd.DataFrame({
    "income": income,
    "credit_score": credit_score,
    "loan_amount": loan_amount,
    "term_months": term_months,
    "dti_ratio": dti_ratio,
    "education_level": education_level,
    "loan_purpose": loan_purpose,
    "default": default
})

print("Dataset shape:", df.shape)
print("Default rate:", df["default"].mean())
df.head()


In [ ]:
# Cell 2: Split (BUG: wrong test_size), raw categoricals, train model
feature_cols = [
    "income", "credit_score", "loan_amount", "term_months", "dti_ratio",
    "education_level", "loan_purpose"
]
X = df[feature_cols]   # BUG: no encoding
y = df["default"]

# BUG: should be test_size=0.2
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.75, random_state=42, stratify=y
)

model = RandomForestClassifier(n_estimators=200, max_depth=8, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# BUG: macro average is wrong for binary here
precision = precision_score(y_test, y_pred, average="macro")
recall = recall_score(y_test, y_pred, average="macro")

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# BUG: missing F1, feature importances dict, predict_proba, encoding
model_metrics = {"accuracy": accuracy, "precision": precision, "recall": recall}
model_metrics
